## Deliverable 2. Create a Customer Travel Destinations Map.

In [ ]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps



In [ ]:
# Import API key
from config import gmap_key

# Configure gmaps API key
gmaps.configure(api_key=gmap_key)

In [ ]:
# 1. Import the WeatherPy_database.csv file. 
import os
file_to_load = os.path.join("../Weather_Database", "Weather_Database.csv")
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Rikitea,PF,2020-08-30 16:32:03,-23.12,-134.97,69.31,83,65,19.89,light rain
5,5,Bethel,US,2020-08-30 16:32:03,41.37,-73.41,72.00,50,19,1.01,few clouds
9,9,Waingapu,ID,2020-08-30 16:24:30,-9.66,120.26,74.57,79,78,12.53,broken clouds
13,13,Oktyabrskoye,RU,2020-08-30 16:24:30,43.06,44.74,73.00,72,20,6.71,few clouds
16,16,Hilo,US,2020-08-30 16:30:14,19.73,-155.09,71.60,83,75,3.36,broken clouds
18,18,Winnemucca,US,2020-08-30 16:20:17,40.97,-117.74,71.60,14,1,12.75,clear sky
30,30,Albany,US,2020-08-30 16:32:07,42.60,-73.97,71.01,49,24,3.00,few clouds
36,36,Morgantown,US,2020-08-30 16:23:46,39.63,-79.96,71.60,53,40,4.70,scattered clouds
41,41,Bambous Virieux,MU,2020-08-30 16:32:08,-20.34,57.76,72.00,78,40,16.11,shower rain
47,47,Shatsk,RU,2020-08-30 16:20:04,54.03,41.71,68.61,66,41,9.80,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                108
City                   108
Country                106
Date                   108
Lat                    108
Lng                    108
Max Temp               108
Humidity               108
Cloudiness             108
Wind Speed             108
Current Description    108
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,69.31,light rain,-23.12,-134.97,
5,Bethel,US,72.00,few clouds,41.37,-73.41,
9,Waingapu,ID,74.57,broken clouds,-9.66,120.26,
13,Oktyabrskoye,RU,73.00,few clouds,43.06,44.74,
16,Hilo,US,71.60,broken clouds,19.73,-155.09,
18,Winnemucca,US,71.60,clear sky,40.97,-117.74,
30,Albany,US,71.01,few clouds,42.60,-73.97,
36,Morgantown,US,71.60,scattered clouds,39.63,-79.96,
41,Bambous Virieux,MU,72.00,shower rain,-20.34,57.76,
47,Shatsk,RU,68.61,scattered clouds,54.03,41.71,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gmap_key
}

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head()    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,69.31,light rain,-23.12,-134.97,Pension Maro'i
5,Bethel,US,72.00,few clouds,41.37,-73.41,Courtyard by Marriott Danbury
9,Waingapu,ID,74.57,broken clouds,-9.66,120.26,Pondok Wisata Elim
13,Oktyabrskoye,RU,73.00,few clouds,43.06,44.74,Prolesok Mini-hotel
16,Hilo,US,71.60,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F and {Current Description} </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.9)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))